In [ ]:
# Comparison among PCA, EFA, and Autoencoder on MSE and EVR
import os

seed = 52
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

print(">>> Global env + basic seeds set, seed =", seed)
import random
import numpy as np
import torch
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
import torch
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from model_code import AE, set_deterministic, _total_evr_from_recon
from validators import build_10_items_validators, build_validators_baseline, build_model_scores


>>> Global env + basic seeds set, seed = 52
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
code_dir = Path.cwd()
data_path = code_dir.parent / 'data'
data_file = data_path / 'cbcl_data_remove_low_frequency.csv'
if not data_file.exists():
    raise FileNotFoundError(f'Could not find {data_file}')

qns = pd.read_csv(data_file, encoding='utf-8')
X = qns.iloc[:, 1:].values


X_train_raw, X_temp = train_test_split(X, test_size=0.2, random_state=seed)
X_val_raw, X_test_raw = train_test_split(X_temp, test_size=0.5, random_state=seed)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_raw)
X_val = scaler.transform(X_val_raw)
X_test = scaler.transform(X_test_raw)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_factors = 5


In [6]:
fa = FactorAnalyzer(n_factors=n_factors, rotation='geomin_obl')
fa.fit(X_scaled)
efa_scores = fa.transform(X_scaled)
X_recon_efa = efa_scores @ fa.loadings_.T
mse_efa = float(np.mean((X_scaled - X_recon_efa) ** 2))
evr_efa = _total_evr_from_recon(X_scaled, X_recon_efa)

pca = PCA(n_components=n_factors)
pca_scores = pca.fit_transform(X_scaled)
X_recon_pca = pca.inverse_transform(pca_scores)
mse_pca = float(np.mean((X_scaled - X_recon_pca) ** 2))
evr_pca = _total_evr_from_recon(X_scaled, X_recon_pca)

print(f'EFA -> MSE={mse_efa:.6f}, Total EVR={evr_efa:.4f}')
print(f'PCA -> MSE={mse_pca:.6f}, Total EVR={evr_pca:.4f}')


EFA -> MSE=0.737914, Total EVR=0.2621
PCA -> MSE=0.728905, Total EVR=0.2711


In [8]:
set_deterministic(seed)

autoencoder = AE(
    X_train,
    X_val,
    encoding_dim=5,
    h1=192,
    h2=64,
    seed=seed,
    lr=0.003,
    lambda_decorr=0.005,
    weight_decay=8e-05,
)
autoencoder.train(show_plot=False)

latent_factors, _, _, evr_test, reconstructed, _ = autoencoder.evaluate_on_data(X_test)
print(f"Evaluating on test set: EVR test = {evr_test:.4f}")

latent_factors, _, _, evr_total, reconstructed, _ = autoencoder.evaluate_on_data(X_scaled)
mse_ae = float(np.mean((X_scaled - reconstructed) ** 2))

print(f'AE -> MSE={mse_ae:.6f}, Total EVR={evr_total:.4f}')


[Seed fixed] random/numpy/torch all set to 52
Using device: cuda
Early stopping at epoch 100
Evaluating on test set: EVR test = 0.3718
AE -> MSE=0.579299, Total EVR=0.4208


In [9]:
comparison = pd.DataFrame(
    [
        {'model': 'EFA', 'mse': mse_efa, 'total_evr': evr_efa},
        {'model': 'PCA', 'mse': mse_pca, 'total_evr': evr_pca},
        {'model': 'Autoencoder', 'mse': mse_ae, 'total_evr': evr_total},
    ]
)
comparison


,model,mse,total_evr
0,EFA,0.737914,0.262086
1,PCA,0.728905,0.271095
2,Autoencoder,0.579299,0.420823


Compare validations

In [ ]:

ROOT = Path(r"G:\ABCD\abcd-data-release-5.1")
DICT = Path(r"G:\ABCD\datadict51.xlsx")

validators = {
    "sex"   : ["pubertal_sex_p"],
    "site" : ["site_id_l"],
    "age"   : ["interview_age"],
}

# ======== 2│运行并输出 ========
out_frames, wide = build_validators_baseline(
    root=ROOT,
    dict_path=DICT,
    validators=validators,
    eventname="baseline_year_1_arm_1",
    out_dir=None,            # 会输出各 tag 的 *_baseline.csv
    dict_sheet=None,                      # 若 datadict 只有一个 sheet，可不用填
    dict_engine="openpyxl",
    verbose=True,
    wide_table_name="covariates_baseline.csv"
)

# wide 就是 validators_baseline.csv 的 DataFrame 对象
print(wide.shape, wide.columns[:10])

[INFO] Reading data dictionary …

=== sex ===
  reading ph_p_pds.csv ...
  -> generated sex in memory  (11868 rows, 2 columns)

=== site ===
  reading abcd_y_lt.csv ...
  -> generated site in memory  (11868 rows, 2 columns)

=== age ===
  reading abcd_y_lt.csv ...
  -> generated age in memory  (11868 rows, 2 columns)
(11868, 4) Index(['src_subject_id', 'pubertal_sex_p', 'site_id_l', 'interview_age'], dtype='object')


In [12]:
AE_scores = build_model_scores(qns, latent_factors)
# combine wide and AE_scores on src_subject_id
validators_df = build_10_items_validators(validators_csv = "../output/validators/validators_baseline.csv")

nn_result = AE_scores.merge(
    validators_df,
    on="src_subject_id",
    how="inner"
)

In [13]:
subject_id = qns.iloc[:, 0]
efa_scores_df = pd.DataFrame(efa_scores, columns=[f"efa_{i+1}" for i in range(efa_scores.shape[1])])
efa_scores_df.insert(0, "src_subject_id", subject_id)

efa_result = efa_scores_df.merge(
    validators_df,
    on="src_subject_id",
    how="inner"
)
# efa_result

In [14]:
subject_id = qns.iloc[:, 0]
pca_scores_df = pd.DataFrame(pca_scores, columns=[f"pca_{i+1}" for i in range(pca_scores.shape[1])])
pca_scores_df.insert(0, "src_subject_id", subject_id)
pca_result = pca_scores_df.merge(
    validators_df,
    on="src_subject_id",
    how="inner"
)
# pca_result

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

def standardize(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0, ddof=1).replace(0, 1)
    return (X - mean) / std

for validator_used in ["dev_delay", "fes_conflict", "n_friends", "school_conn", "avg_grades", 
                       "fluid_cog", "cryst_cog", "mh_service", "med_history", "brought_meds"]:

    # ===== 1. 取 EFA 因子 =====
    tmp = efa_result[[f"efa_{f}" for f in range(1, 5)] + [validator_used]].dropna()
    X_efa = tmp[[f"efa_{f}" for f in range(1, 5)]]
    y = tmp[validator_used]

    X_efa_scaled = standardize(X_efa)

    # ===== 2. 取 AE latent（假设 ae_result 有 ae_1...ae_5）=====
    tmp_ae = nn_result[[f"factor_{i}" for i in range(1, 5)] + [validator_used]].dropna()
    X_ae = tmp_ae[[f"factor_{i}" for i in range(1, 5)]]
    y_ae = tmp_ae[validator_used]
    X_ae_scaled = standardize(X_ae)

    tmp_pca = pca_result[[f"pca_{f}" for f in range(1, 5)] + [validator_used]].dropna()
    X_pca = tmp_pca[[f"pca_{f}" for f in range(1, 5)]]
    y_pca = tmp_pca[validator_used]

    X_pca_scaled = standardize(X_pca)

    # 保证 y / y_ae 对齐的话，可以先 merge 再分，但这里先简写思路

    kf = KFold(n_splits=10, shuffle=True)

    # 线性模型
    lin = LinearRegression()
    r2_efa_lin = cross_val_score(lin, X_efa_scaled, y, cv=kf, scoring="r2").mean()
    r2_ae_lin  = cross_val_score(lin, X_ae_scaled, y_ae, cv=kf, scoring="r2").mean()
    r2_pca_lin = cross_val_score(lin, X_pca_scaled, y_pca, cv=kf, scoring="r2").mean()

    # 非线性：MLP
    model = MLPRegressor(hidden_layer_sizes=(32, 16),
                    activation="relu",
                    random_state=seed,
                    max_iter=1000)
    # model = RandomForestRegressor(n_estimators=100,
    #                               max_depth=5)
    r2_efa_mlp = cross_val_score(model, X_efa_scaled, y, cv=kf, scoring="r2").mean()
    r2_ae_mlp  = cross_val_score(model, X_ae_scaled, y_ae, cv=kf, scoring="r2").mean()
    r2_pca_mlp = cross_val_score(model, X_pca_scaled, y_pca, cv=kf, scoring="r2").mean()
    
    print(f"Validator: {validator_used}")
    print(f"EFA  - Linear: {r2_efa_lin:.3f}, MLP: {r2_efa_mlp:.3f}")
    print(f"AE   - Linear: {r2_ae_lin:.3f}, MLP: {r2_ae_mlp:.3f}")
    print(f"PCA  - Linear: {r2_pca_lin:.3f}, MLP: {r2_pca_mlp:.3f}")

Validator: dev_delay
EFA  - Linear: 0.018, MLP: 0.013
AE   - Linear: 0.009, MLP: 0.016
PCA  - Linear: 0.023, MLP: 0.022
Validator: fes_conflict
EFA  - Linear: 0.029, MLP: 0.038
AE   - Linear: 0.023, MLP: 0.026
PCA  - Linear: 0.030, MLP: 0.026
Validator: n_friends
EFA  - Linear: 0.011, MLP: 0.012
AE   - Linear: 0.006, MLP: 0.003
PCA  - Linear: 0.014, MLP: 0.015
Validator: school_conn
EFA  - Linear: 0.023, MLP: 0.020
AE   - Linear: 0.022, MLP: 0.015
PCA  - Linear: 0.023, MLP: 0.017
Validator: avg_grades
EFA  - Linear: 0.143, MLP: 0.155
AE   - Linear: 0.101, MLP: 0.115
PCA  - Linear: 0.143, MLP: 0.156
Validator: fluid_cog
EFA  - Linear: 0.049, MLP: 0.058
AE   - Linear: 0.031, MLP: 0.038
PCA  - Linear: 0.050, MLP: 0.056
Validator: cryst_cog
EFA  - Linear: 0.050, MLP: 0.057
AE   - Linear: 0.027, MLP: 0.044
PCA  - Linear: 0.049, MLP: 0.053
Validator: mh_service
EFA  - Linear: 0.167, MLP: 0.161
AE   - Linear: 0.119, MLP: 0.149
PCA  - Linear: 0.162, MLP: 0.155
Validator: med_history
EFA  - Lin